Instrumented difference in difference

Potential Covarites:
  - mental health
  - obesity rates
  - tobacco use(# of Cigarettes, percentage of Cigarettes use)
  - prescription drug use by states



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.dates as mdates
import os
%config InlineBackend.figure_format = 'retina'
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression


# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set directory
os.chdir("/content/drive/MyDrive/econ191/raw/cdc")

**Data Cleaning for BRFSS datasets**
  - *Year Between 2003 and 2018*

In [ ]:
# Year 2003
df = pd.read_csv("brfss03.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78) | (df['x.state'] == 66)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height']
weight = df['weight']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthplan']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokeday']

# Alcohol
alcohol_freq = df['alcday3']
alcohol_avg = df['avedrnk']
nof_alcohol_day = df['x.drnkdy2']


income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}
marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)
mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2003'+'clean.csv')

ValueError: Length mismatch: Expected axis has 22 elements, new values have 23 elements

In [ ]:
# Year 2004
df = pd.read_csv("brfss04.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height2']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthplan']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokeday']

# Alcohol
alcohol_freq = df['alcday3']
alcohol_avg = df['avedrnk']
nof_alcohol_day = df['x.drnkdy2']


income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight','bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2004'+'clean.csv')

In [ ]:
# Year 2005
df = pd.read_csv("brfss05.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthplan']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday4']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy3']


income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight','bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2005'+'clean.csv')

In [ ]:
# Year 2006
df = pd.read_csv("brfss06.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthplan']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday4']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy3']


income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight','bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2006'+'clean.csv')

In [ ]:
# Year 2007
df = pd.read_csv("brfss07.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthplan']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday4']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy3']

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight','bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2007'+'clean.csv')

In [ ]:
# Year 2008
df = pd.read_csv("brfss08.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthplan']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday4']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy3']

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight','bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2008'+'clean.csv')

In [ ]:
# Year 2009
df = pd.read_csv("brfss09.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthplan']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday4']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy3']

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2009'+'clean.csv')

In [ ]:
# Year 2010
df = pd.read_csv("brfss10.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthplan']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday4']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy3']

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2010'+'clean.csv')

In [ ]:
# Year 2011
df = pd.read_csv("brfss11.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthpln1']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday5']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy4']

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2011'+'clean.csv')

In [ ]:
# Year 2012
df = pd.read_csv("brfss12.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['race2']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthpln1']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday5']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy4']

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2012'+'clean.csv')

In [ ]:
# Year 2013
df = pd.read_csv("brfss13.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['x.race']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthpln1']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday5']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy4']

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2013'+'clean.csv')

In [ ]:
# Year 2014
df = pd.read_csv("brfss14.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['x.race']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthpln1']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday5']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkdy4']

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2014'+'clean.csv')

In [ ]:
# Year 2015
df = pd.read_csv("brfss15.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['x.race']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthpln1']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday5']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkwek'] / 7

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2015'+'clean.csv')

In [ ]:
# Year 2016
df = pd.read_csv("brfss16.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['x.race']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthpln1']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday5']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkwek'] / 7

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2016'+'clean.csv')

In [ ]:
# Year 2017
df = pd.read_csv("brfss17.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['x.race']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthpln1']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday5']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkwek'] / 7

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2017'+'clean.csv')

In [ ]:
# Year 2018
df = pd.read_csv("brfss18.csv",encoding='cp1252')

# State with 72: Puerto Rico, Virgin Islands drop
rows_to_drop = df[(df['x.state'] == 72) | (df['x.state'] == 78)].index

# Drop the rows using the index values
df.drop(rows_to_drop, inplace=True)

# Age eligibility for mediciad expansion under the ACA(19-64)
# Including 18-24 due to the sanitiy check
df = df[df['x.ageg5yr'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])]

# Demographics
income = df['income2']
race = df['x.race']
state = df['x.state']
age = df['x.ageg5yr']
sex = df['sex1']
height = df['height3']
weight = df['weight2']
year = df['iyear'].astype(int)
educa =  df['educa']
children = df['children']
adult = df['numadult']
marital = df['marital']

# Health Outcomes
general_health = df['genhlth']
physical_health = df['physhlth']
mental_health = df['menthlth']
poor_health = df['poorhlth']

# Health care coverage(health insurance, pre-paid plans Medicare)
health_access = df['hlthpln1']

# Tobacco use
tobacco_100 = df['smoke100']
tobacco_day = df['smokday2']

# Alcohol
alcohol_freq = df['alcday5']
alcohol_avg = df['avedrnk2']
nof_alcohol_day = df['x.drnkwek'] / 7

income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'50k-75k', 8:'>75k', 77:np.nan, 99:np.nan}
race_replace = {1:'white', 2:'black', 3:'asian/non-hispanic', 4:'native hawaiian/pacific islander', 5:'american indian/alaskan native', 6:'other/non-hispanic', 7:'non-hispanic/multiracial', 8:'hispanic', 9:'refused/unknown'}
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {2:0}
hw_replace = {777:np.nan, 999:np.nan}
educa_replace = {1:0,2:0,3:0,4:0,5:1,6:1,9:np.nan}
children_replace = {88:np.nan,99:np.nan}
adult_replace = {88:np.nan,99:np.nan}

marital_replace = {1:'Married', 6:'Cohabitating', 2:'Divorced', 4:'Separated', 3:'Widowed', 5:'Never married', 9:np.nan}
general_health_replace = {1:'Excellent', 2:'Very good',3:'Good',4:'Fair',5:'Poor',7:np.nan,9:'refused/unknown'}
physical_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
mental_health_replace = {88:np.nan, 77:np.nan, 99:np.nan}
poor_health_replace = {88:np.nan, 77:np.nan, 99:np.nan} # Missing Values huge
health_access_replace = {2:0,7:np.nan, 9:np.nan}
tobacco_100_replace = {2:0,7:np.nan, 99:np.nan}
tobacco_day_replace = {1:'Every day', 2:'Some days', 3: 'Not at all',9:'refused/unknown'}
alcohol_freq_replace = {777:np.nan,888:0 ,999:np.nan}
alcohol_avg_replace = {77:np.nan, 99:np.nan}


metric = False
max_weight = 777

income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
height = height.replace(hw_replace)
weight = weight.replace(hw_replace)
educa = educa.replace(educa_replace)
children = children.replace(children_replace)
adult = adult.replace(adult_replace)
marital  = marital.replace(marital_replace)

mental_health = mental_health.replace(mental_health_replace)
general_health = general_health.replace(general_health_replace)
physical_health = physical_health.replace(physical_health_replace)
poor_health = poor_health.replace(poor_health_replace)
health_access = health_access.replace(health_access_replace)
tobacco_100 = tobacco_100.replace(tobacco_100_replace)
tobacco_day = tobacco_day.replace(tobacco_day_replace)
alcohol_freq = alcohol_freq.replace(alcohol_freq_replace)
alcohol_avg = alcohol_avg.replace(alcohol_avg_replace)

height = height.tolist()
new_height = []
for row in height:
    h = str(row)

    if row < 1:
        meters = np.nan

    elif row < 712:
        feet = float(h[0])
        inches = float(h[1:])
        if inches > 12:
            meters = np.nan
        else:
            inches = inches + feet*12
            meters = inches * 0.0254

    elif row < 9999 and row >= 9000 and metric:
        meters = float(h[1])+float(h[2:])*0.01
        if meters == 0:
            meters = np.nan

    else:
        meters = np.nan

    new_height.append(meters)

weight = weight.tolist()
new_weight = []
for row in weight:

    if row < 10:
        kg = np.nan

    elif row < max_weight:
        kg = row * 0.453592

    elif row < 9999 and metric:
        w = str(row)
        kg = float(w[1:])
        if kg < 10:
            kg = np.nan

    else:
        kg = np.nan

    new_weight.append(kg)

bmi = []
for h, w in zip(new_height, new_weight):
    b = w/(h*h)
    if b < 10 or b > 200:
        b = np.nan
    bmi.append(b)

height = pd.Series(new_height)
weight = pd.Series(new_weight)
bmi = pd.Series(bmi)

brfss_out = pd.concat([year,income,educa, children, adult,marital,race, state, age, sex, height, weight, bmi, mental_health, general_health,physical_health,poor_health,health_access,tobacco_100,tobacco_day,alcohol_freq,alcohol_avg], axis=1)
brfss_out.columns = ['year','income','educa', 'children','adult','marital','race', 'state', 'age', 'sex', 'height', 'weight', 'bmi','mental_health', 'general_health', 'physical_health','poor_health', 'health_access', 'tobacco_100', 'tobacco_day', 'alcohol_freq', 'alcohol_avg']
brfss_out.dropna(subset=['income','adult','children','general_health','mental_health','physical_health'],inplace = True)
brfss_out.to_csv('/content/drive/MyDrive/econ191/clean/brfss'+'2018'+'clean.csv')

In [ ]:
# Income mapping with midpoints
income_map = {
    '<10k': 5000,
    '10k-15k': 12500,
    '15k-20k': 17500,
    '20k-25k': 22500,
    '25k-35k': 30000,
    '35k-50k': 42500,
    '50k-75k': 62500,
    '>75k': 87500
}

# Function to process a single year's data
def process_year_data(year):
    # Load your DataFrame for the specific year
    # Adjust the file path according to your data organization
    brfss_out = pd.read_csv(f'/content/drive/MyDrive/econ191/clean/cdc/brfss{year}clean.csv')

    # Infer number of adults if missing
    def infer_adults(row):
        if pd.isna(row['adult']):
            return 2 if row['marital'] in ['Married', 'Cohabitating'] else 1
        return row['adult']

    # Apply the inference function
    brfss_out['adult'] = brfss_out.apply(infer_adults, axis=1)

    # Calculate total household size
    brfss_out['household_size'] = brfss_out['adult'] + brfss_out['children'] + 1

    # Replace income categories with numerical midpoints
    brfss_out['income_numeric'] = brfss_out['income'].replace(income_map)

    # Define FPL values for years 2004 to 2018
    fpl_by_year_and_size = {
        2004: {1: 9310, 2: 12490, 3: 15770, 4: 19050},
        2005: {1: 9570, 2: 12830, 3: 16190, 4: 19530},
        2006: {1: 9800, 2: 13200, 3: 16520, 4: 20000},
        2007: {1: 10210, 2: 13760, 3: 17210, 4: 20650},
        2008: {1: 10400, 2: 14000, 3: 17600, 4: 21200},
        2009: {1: 10830, 2: 14570, 3: 18310, 4: 22050},
        2010: {1: 10830, 2: 14570, 3: 18310, 4: 22050},
        2011: {1: 10890, 2: 14710, 3: 18430, 4: 22350},
        2012: {1: 11170, 2: 15130, 3: 19090, 4: 23050},
        2013: {1: 11490, 2: 15510, 3: 19530, 4: 23550},
        2014: {1: 11670, 2: 15730, 3: 19890, 4: 23850},
        2015: {1: 11770, 2: 15930, 3: 20190, 4: 24250},
        2016: {1: 11880, 2: 16020, 3: 20340, 4: 24300},
        2017: {1: 12060, 2: 16240, 3: 20420, 4: 24600},
        2018: {1: 12140, 2: 16460, 3: 20660, 4: 25100}
    }

    # Maximum household size
    max_household_size = brfss_out['household_size'].max()

    # Expand FPL values
    fpl_expanded = {}
    for y, sizes in fpl_by_year_and_size.items():
        fpl_expanded[y] = sizes.copy()
        diffs = {i: sizes[i + 1] - sizes[i] for i in range(1, len(sizes))}
        last_diff = diffs[3]  # use the last difference to extrapolate
        current_size = len(sizes) + 1
        while current_size <= max_household_size:
            fpl_expanded[y][current_size] = fpl_expanded[y][current_size - 1] + last_diff
            current_size += 1

    # Function to get the FPL threshold for a given year and household size
    def get_fpl_threshold(row):
        year = row['year']
        household_size = row['household_size']
        return fpl_expanded.get(year, {}).get(household_size, np.nan)

    # Add a new column for the FPL threshold
    brfss_out['fpl_threshold'] = brfss_out.apply(get_fpl_threshold, axis=1)

    # Compute income as a percentage of the FPL
    brfss_out['income_percent_fpl'] = (brfss_out['income_numeric'] / brfss_out['fpl_threshold']) * 100

    # Example filters
    below_138_fpl = brfss_out[brfss_out['income_percent_fpl'] < 138]
    below_200_fpl = brfss_out[brfss_out['income_percent_fpl'] < 200]
    below_300_fpl = brfss_out[brfss_out['income_percent_fpl'] < 300]
    below_400_fpl = brfss_out[brfss_out['income_percent_fpl'] < 400]
    highest_income_category = brfss_out[brfss_out['income'] == '>75k']

    # Save the cleaned data
    brfss_out.to_csv(f'/content/drive/MyDrive/econ191/clean/cdc/clean2/brfss_{year}_clean.csv', index=False)

# Process each year from 2004 to 2018
for year in range(2004, 2019):
    process_year_data(year)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.dates as mdates
import os
%config InlineBackend.figure_format = 'retina'
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression


# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# As a dataframe
brfss_clean = {}
brfss_clean_analysis = {}

for i in range(2004, 2019):
    brfss_clean[i] = pd.read_csv(f'/content/drive/MyDrive/econ191/clean/cdc/clean2/brfss_{i}_clean.csv')
    brfss_clean[i].drop(columns=['Unnamed: 0'], inplace=True)
    brfss_clean[i] = brfss_clean[i][['year','age','state','income','children','adult','household_size','health_access','general_health','mental_health','physical_health','fpl_threshold','income_percent_fpl']]
    brfss_clean_analysis[i] = brfss_clean[i][brfss_clean[i]['income_percent_fpl'] < 138] # Only for those groups with below 138 fpl

    brfss_clean_analysis[i] = brfss_clean_analysis[i].groupby('state').mean(['mental_health','physical_health','general_health'])

In [ ]:
# Distinguish between Control and Treatment group
# Controlling with demographich, find the mental health indiciator

# Enrollment rate

In [ ]:
nof_enrol = pd.read_csv("/content/drive/MyDrive/econ191/raw/educ/nof_enrollments_fall.csv")
enroll_rates = pd.read_csv("/content/drive/MyDrive/econ191/raw/educ/enrollment_rates_fall.csv")

In [ ]:
enroll_rates.drop(columns=['Unnamed: 19', 'Unnamed: 20'], inplace=True)

In [ ]:
# Check Missing values
nof_enrol